In [98]:
import pandas as pd
import sqlite3 as lite
import cPickle as pickle


####Input aus Excel-Tabellen
#Einlesen der Gen-, Metabolit- und Mediumsspezifikation
genes = pd.read_csv('genes.csv',header=4, index_col=None)
metabolites =pd.read_csv('metabolite.csv',header=4, index_col=0, na_values=['NaN'], keep_default_na=False)
media = pd.read_csv('media.csv',header=3,index_col=0, na_values=['NaN'],keep_default_na=False)
protein_hl = pd.read_csv('mRNA.csv',header=4, index_col=0, na_values=[' '],keep_default_na=False)


#Editieren der Dataframes
#Herauswerfen aller unwichtigen Spalten
metabolites = metabolites['Name']
genes = genes[['BioCyc','SwissProt','GenBank','Name','Sequence','Coordinate']]
media = media['Concentration mM']
protein_hl = protein_hl[['Symbol', 'Protein Halflife']]


#Konkatenieren der Listen an entsprechender Stelle und Herauswerfen der NaN Werte
conc = pd.concat([metabolites, media], axis=1)
conc = conc[pd.notnull(conc['Concentration mM'])]


##Laden des vollstaendigen Genoms in einen Objekt
with open('sequence.fasta','r') as f:
	genome = f.readlines()
f.close()
#Herauswerfen der ersten Zeile und zusammenhaengen der Sequenzen in einen String
genome = [line[:-1] for line in genome if line[0] != '>']
genome = ''.join(genome)



helplist1 = ['genes', 'protein_hl','metabolites','conc']
helplist2 = [genes, protein_hl, metabolites, conc]
helper = dict([helplist1[t], helplist2[t]] for t in xrange(4))
#print helper.keys()
#print helper['genes'].columns


#print '5'+2
#name=genes/metabolites/media
def writeData(data,name):
	f = open(name,'w')
	with f:
		f.write(data)

        


con = lite.connect('./database/knowledgebase.db')
with con:
    cur = con.cursor()    
    cur.execute('SELECT * FROM conc')
    
    data = cur.fetchall()
    #for row in data:
    #    print row
    indexhelp = [data[i][0] for i in xrange(len(data))]
    dframe = pd.DataFrame(data, index=indexhelp, columns=['dump','Name', 'Concentration mM'])
    dframe = dframe.drop('dump',axis=1)
    #print dframe
    print genome
#conc

...


In [ ]:
#